# IBM Watson OpenScale Lab instructions

This notebook should be run in a Watson Studio project, using with **Python 3.5 with Spark** runtime environment. **If you are viewing this in Watson Studio and do not see Python 3.5 with Spark in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following Cloud services:
  * IBM Watson OpenScale
  * Watson Machine Learning
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio and continuous learning services. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

# Package installation

In [2]:
!rm -rf $PIP_BUILD
!pip install psycopg2-binary | tail -n 1
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install ibm-ai-openscale==2.1.1 --no-cache | tail -n 1
!pip install --upgrade numpy --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1

notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
Requirement not upgraded as not directly required: jmespath<1.0.0,>=0.7.1 in /usr/local/src/conda3_runtime.v51/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from ibm-cos-sdk-core==2.*,>=2.0.0->ibm-cos-sdk->watson-machine-learning-client) (0.9.3)
notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 6.0.0 requires widgetsnbextension~=2.0.0, which is not installed.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
notebook 5.0.0 requires nbconvert, which is not installed.
ipywidgets 

# Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users).  
* From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. 
* Give your key a name and click **Create**, then copy the created key and paste it below.

Alternately, from the [IBM Cloud CLI](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use) :

```bash
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

In [3]:
CLOUD_API_KEY = "*****"

Get the Watson OpenScale GUID using the [IBM Cloud CLI](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use) :

```bash
ibmcloud resource service-instance <Watson_OpenScale_instance_name>
```

In [4]:
AIOS_GUID = "*****"

In [5]:
AIOS_CREDENTIALS = {
    "instance_guid": AIOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

Next you will need credentials for Watson Machine Learning. If you already have a WML instance, you may use credentials for it. To provision a new Lite instance of WML, use the [IBM Cloud catalog](https://cloud.ibm.com/catalog/services/machine-learning):  
* Give your service a name, and click **Create**.  
* Once your instance is created, click the **Service Credentials** link on the left side of the screen.  
* Click the **New credential** button, give your credentials a name, and click **Add**.  
* Your new credentials can be accessed by clicking the **View credentials** button.  
* Copy and paste your WML credentials into the cell below.

In [6]:
WML_CREDENTIALS = {
  "apikey": "kjfkdjfkdjfkdjkdjfkdjfkdfj",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/fkjkfdkfjkdjkdjfkdjfkdfjdkfabb::",
  "iam_apikey_name": "auto-generated-apikey-kdjfkjdkdjf99938493893e3",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/kdfjkdjfkdjfkd9ee42::serviceid:ServiceId-kjfkdejfkdjfkdjfkdjfd54",
  "instance_id": "kfjkdjfkdjfdkfjde66abb",
  "password": "kdfjkdfjkdjfkdjf21e0",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "kejfkdjfkdjfkdfjdkjfe3"
}


This lab can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

**If you have previously configured OpenScale**, it will use your existing datamart, and not interfere with any models you are currently monitoring. **Do not update the cell below**.

**If you do not have a paid Cloud account or would prefer not to provision this paid service**, you may use the free internal PostgreSQL service with OpenScale. **Do not update the cell below**.

**To provision a new instance of Db2 Warehouse**:  
* Locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse)  
* Give your service a name, and click **Create**.  
* Once your instance is created, click the **Service Credentials** link on the left side of the screen.  
* Click the **New credential** button, give your credentials a name, and click **Add**.  
* Your new credentials can be accessed by clicking the **View credentials** button.  
* Copy and paste your Db2 Warehouse credentials into the cell below.

**To provision a new instance of Databases for PostgreSQL**:
* Locate [Databases for PostgreSQL in the Cloud catalog](*https://cloud.ibm.com/catalog/services/databases-for-postgresql)  
* Give your service a name, and click **Create**.  
* Once your instance is created, click the **Service Credentials** link on the left side of the screen.  
* Click the **New credential** button, give your credentials a name, and click **Add**.  
* Your new credentials can be accessed by clicking the **View credentials** button.  
* Copy and paste your Databases for PostgreSQL credentials into the cell below.

In [28]:
DB_CREDENTIALS = {
  "hostname": "***",
  "password": "***",
  "https_url": "***",
  "port": 50000,
  "ssldsn": "***",
  "host": "***",
  "jdbcurl": "***",
  "uri": "***",
  "db": "***",
  "dsn": "****",
  "username": "***",
  "ssljdbcurl": "****"
}

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [7]:
KEEP_MY_INTERNAL_POSTGRES = True

# Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Load and explore data

## Load the training data from github

In [7]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_data_biased_training.csv

--2019-03-18 14:33:51--  https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

100%[======================================>] 689,622     --.-K/s   in 0.03s   

2019-03-18 14:33:51 (22.3 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [8]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

Row(CheckingStatus='0_to_200', LoanDuration=31, CreditHistory='credits_paid_to_date', LoanPurpose='other', LoanAmount=1889, ExistingSavings='100_to_500', EmploymentDuration='less_1', InstallmentPercent=3, Sex='female', OthersOnLoan='none', CurrentResidenceDuration=3, OwnsProperty='savings_insurance', Age=32, InstallmentPlans='none', Housing='own', ExistingCreditsCount=1, Job='skilled', Dependents=1, Telephone='none', ForeignWorker='yes', Risk='No Risk')

## Explore data

In [9]:
df_data.printSchema()

root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)



In [10]:
print("Number of records: " + str(df_data.count()))

Number of records: 5000


# Create a model

In [15]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Spark German Risk Model - Final"
DEPLOYMENT_NAME = "Spark German Risk Deployment - Final"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

Number of records for training: 4016
Number of records for evaluation: 984
root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = 

In [16]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model

si_CheckingStatus = StringIndexer(inputCol = 'CheckingStatus', outputCol = 'CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol = 'CreditHistory', outputCol = 'CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol = 'LoanPurpose', outputCol = 'LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol = 'ExistingSavings', outputCol = 'ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol = 'EmploymentDuration', outputCol = 'EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol = 'Sex', outputCol = 'Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol = 'OthersOnLoan', outputCol = 'OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol = 'OwnsProperty', outputCol = 'OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol = 'InstallmentPlans', outputCol = 'InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol = 'Housing', outputCol = 'Housing_IX')
si_Job = StringIndexer(inputCol = 'Job', outputCol = 'Job_IX')
si_Telephone = StringIndexer(inputCol = 'Telephone', outputCol = 'Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol = 'ForeignWorker', outputCol = 'ForeignWorker_IX')

In [17]:
si_Label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [18]:
va_features = VectorAssembler(inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX", "EmploymentDuration_IX", "Sex_IX", \
                                         "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX", "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", \
                                         "LoanDuration", "LoanAmount", "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount", \
                                         "Dependents"], outputCol="features")

In [19]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker, si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,\
                               si_OwnsProperty, si_Sex, si_Telephone, si_Label, va_features, classifier, label_converter])
model = pipeline.fit(train_data)

In [25]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.704369


# Save and deploy the model

In [21]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### Remove existing model and deployment

In [22]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

Deleting deployment id 71a0bee8-a90f-4bf2-bca0-8c390e8dba67
Deleting model id 4ca90750-43d3-4535-9052-170ce8616675
------------------------------------  --------------  ------------------------  ---------
GUID                                  NAME            CREATED                   FRAMEWORK
52dc0b30-606f-40ec-b3de-f67d67acf6f9  drug-selection  2018-11-07T20:22:33.177Z  mllib-2.1
------------------------------------  --------------  ------------------------  ---------


In [26]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": area_under_curve,
           "threshold": 0.7
        }
    ]
}

In [27]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

Storing model ...
Done


In [28]:
model_uid

'b9473da0-9ae7-474a-a397-2b6ce7aa6b65'

In [29]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: 'b9473da0-9ae7-474a-a397-2b6ce7aa6b65' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4d125552-83ee-456a-b1d4-3fd518743082'
------------------------------------------------------------------------------------------------


Model id: b9473da0-9ae7-474a-a397-2b6ce7aa6b65
Deployment id: 4d125552-83ee-456a-b1d4-3fd518743082


# Configure OpenScale

In [30]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

## Create schema and datamart

In [31]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version

'2.1.1'

#### Note: to delete an existing data_mart, uncomment the next cell and run it.

In [25]:
#ai_client.data_mart.delete()

### Set up datamart

In [32]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        try:
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
        except:
            print('Setup failed, trying Db2 setup')
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS, schema=DB_CREDENTIALS['username'])

Using existing internal datamart.


In [33]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

{'database_configuration': {},
 'internal_database': True,
 'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/46816354d9cb773bae86c226aa74c8cc:f94e20cb-aa92-4636-97ce-08de6b714d02::',
 'status': {'state': 'active'}}

## Bind machine learning engines

In [34]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

9e7215a4-16bb-464f-9e31-ff9270e66abb,WML instance,watson_machine_learning,2019-03-19T14:02:36.415Z


In [35]:
print(binding_uid)

9e7215a4-16bb-464f-9e31-ff9270e66abb


In [36]:
ai_client.data_mart.bindings.list_assets()

b9473da0-9ae7-474a-a397-2b6ce7aa6b65,Spark German Risk Model - Final,2019-03-19T14:00:27.876Z,model,mllib-2.1,9e7215a4-16bb-464f-9e31-ff9270e66abb,False
52dc0b30-606f-40ec-b3de-f67d67acf6f9,drug-selection,2018-11-07T20:22:43.078Z,model,mllib-2.1,9e7215a4-16bb-464f-9e31-ff9270e66abb,False


## Subscriptions

### Remove existing credit risk subscriptions

In [37]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

In [38]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Risk',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [39]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

b9473da0-9ae7-474a-a397-2b6ce7aa6b65,Spark German Risk Model - Final,model,9e7215a4-16bb-464f-9e31-ff9270e66abb,2019-03-19T14:02:56.917Z


In [40]:
subscription.get_details()

{'entity': {'asset': {'asset_id': 'b9473da0-9ae7-474a-a397-2b6ce7aa6b65',
   'asset_type': 'model',
   'created_at': '2019-03-19T14:00:27.876Z',
   'name': 'Spark German Risk Model - Final',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/9e7215a4-16bb-464f-9e31-ff9270e66abb/published_models/b9473da0-9ae7-474a-a397-2b6ce7aa6b65'},
  'asset_properties': {'categorical_fields': ['CheckingStatus',
    'CreditHistory',
    'LoanPurpose',
    'ExistingSavings',
    'EmploymentDuration',
    'Sex',
    'OthersOnLoan',
    'OwnsProperty',
    'InstallmentPlans',
    'Housing',
    'Job',
    'Telephone',
    'ForeignWorker'],
   'feature_fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'J

### Score the model so we can configure monitors

In [41]:
credit_risk_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        credit_risk_scoring_endpoint = deployment['entity']['scoring_url']
        
print(credit_risk_scoring_endpoint)

4d125552-83ee-456a-b1d4-3fd518743082
https://us-south.ml.cloud.ibm.com/v3/wml_instances/9e7215a4-16bb-464f-9e31-ff9270e66abb/deployments/4d125552-83ee-456a-b1d4-3fd518743082/online


In [42]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)

print(scoring_response)

{'fields': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, [21, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 1.0, 1.0, 13.0, 1343.0, 2.0, 3.0, 13.0, 

## Quality and feedback monitoring

### Enable quality monitoring

Wait ten seconds to allow the payload logging table to be set up before we begin enabling monitors.

In [43]:
time.sleep(10)
subscription.quality_monitoring.enable(threshold=0.7, min_records=50)

### Feedback logging

In [44]:
!rm additional_feedback_data.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/additional_feedback_data.json

--2019-03-19 09:04:02--  https://raw.githubusercontent.com/emartensibm/german-credit/master/additional_feedback_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16506 (16K) [text/plain]
Saving to: ‘additional_feedback_data.json’

100%[======================================>] 16,506      --.-K/s   in 0.001s  

2019-03-19 09:04:02 (27.8 MB/s) - ‘additional_feedback_data.json’ saved [16506/16506]



In [45]:
with open('additional_feedback_data.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)
subscription.feedback_logging.store(additional_feedback_data['data'])

In [46]:
subscription.feedback_logging.show_table()

less_0,10,all_credits_paid_back,car_new,250,500_to_1000,4_to_7,3,male,none,2,real_estate,23,none,rent,1,skilled,1,none,yes,No Risk,2019-03-19 14:04:13.230000+00:00
no_checking,23,prior_payments_delayed,appliances,6964,100_to_500,4_to_7,4,female,none,3,car_other,39,none,own,1,skilled,1,none,yes,Risk,2019-03-19 14:04:13.230000+00:00
0_to_200,30,outstanding_credit,appliances,3464,100_to_500,greater_7,3,male,guarantor,4,savings_insurance,51,stores,free,1,skilled,1,yes,yes,Risk,2019-03-19 14:04:13.230000+00:00
no_checking,23,outstanding_credit,car_used,2681,500_to_1000,greater_7,4,male,none,3,car_other,33,stores,free,1,unskilled,1,yes,yes,No Risk,2019-03-19 14:04:13.230000+00:00
0_to_200,18,prior_payments_delayed,furniture,1673,less_100,1_to_4,2,male,none,3,car_other,30,none,own,2,skilled,1,none,yes,Risk,2019-03-19 14:04:13.230000+00:00
no_checking,44,outstanding_credit,radio_tv,3476,unknown,greater_7,4,male,co-applicant,4,unknown,60,none,free,2,skilled,2,yes,yes,Risk,2019-03-19 14:04:13.230000+00:00
less_0,8,no_credits,education,803,less_100,unemployed,1,male,none,1,savings_insurance,19,stores,rent,1,skilled,1,none,yes,No Risk,2019-03-19 14:04:13.230000+00:00
0_to_200,7,all_credits_paid_back,car_new,250,less_100,unemployed,1,male,none,1,real_estate,19,stores,rent,1,skilled,1,none,yes,No Risk,2019-03-19 14:04:13.230000+00:00
0_to_200,33,credits_paid_to_date,radio_tv,3548,100_to_500,1_to_4,3,male,none,4,car_other,28,none,own,2,skilled,1,yes,yes,Risk,2019-03-19 14:04:13.230000+00:00
no_checking,24,prior_payments_delayed,retraining,4158,100_to_500,greater_7,3,female,none,2,savings_insurance,35,stores,own,1,unskilled,2,none,yes,Risk,2019-03-19 14:04:13.230000+00:00


In [47]:
run_details = subscription.quality_monitoring.run()
status = run_details['status']
id = run_details['id']
print(id)

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

b5695cca-2a27-49e3-b2ec-e5be44bca944
Run status: running
Run status: running
Run status: completed


In [48]:
subscription.quality_monitoring.get_run_details()

{'evaluations': [{'completed_at': '2019-03-19T14:04:54.099Z',
   'data_mart_id': 'f94e20cb-aa92-4636-97ce-08de6b714d02',
   'flags': {'batch_size': 1000,
    'fetch_size': 1000,
    'force': False,
    'parallelism': 4,
    'window_in_millis': 2000},
   'id': 'b5695cca-2a27-49e3-b2ec-e5be44bca944',
   'metrics': {'area_under_pr': 0.33858998144712427,
    'area_under_roc': 0.7469512195121951},
   'problem_type': 'binary',
   'rows_in_range': 98,
   'rows_max': 10000,
   'rows_min': 50,
   'service_binding_id': '9e7215a4-16bb-464f-9e31-ff9270e66abb',
   'stages': [{'completed_at': '2019-03-19T14:04:42.800Z',
     'id': 1,
     'name': 'Prerequisite Check',
     'properties': {'input_columns': ['CheckingStatus',
       'LoanDuration',
       'CreditHistory',
       'LoanPurpose',
       'LoanAmount',
       'ExistingSavings',
       'EmploymentDuration',
       'InstallmentPercent',
       'Sex',
       'OthersOnLoan',
       'CurrentResidenceDuration',
       'OwnsProperty',
       'Age'

In [49]:
subscription.quality_monitoring.show_table()

2019-03-19 14:04:38.531000+00:00,0.7469512195121951,0.7,9e7215a4-16bb-464f-9e31-ff9270e66abb,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,4d125552-83ee-456a-b1d4-3fd518743082,Accuracy_evaluation_b5695cca-2a27-49e3-b2ec-e5be44bca944,


In [50]:
subscription.quality_monitoring._get_data_from_rest_api()

[[datetime.datetime(2019, 3, 19, 14, 4, 38, 531000, tzinfo=tzutc()),
  0.7469512195121951,
  0.7,
  '9e7215a4-16bb-464f-9e31-ff9270e66abb',
  'b9473da0-9ae7-474a-a397-2b6ce7aa6b65',
  '4d125552-83ee-456a-b1d4-3fd518743082',
  'Accuracy_evaluation_b5695cca-2a27-49e3-b2ec-e5be44bca944',
  '']]

In [51]:
ai_client.data_mart.get_deployment_metrics()

{'deployment_metrics': [{'asset': {'asset_id': 'b9473da0-9ae7-474a-a397-2b6ce7aa6b65',
    'asset_type': 'model',
    'created_at': '2019-03-19T14:00:27.876Z',
    'name': 'Spark German Risk Model - Final',
    'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/9e7215a4-16bb-464f-9e31-ff9270e66abb/published_models/b9473da0-9ae7-474a-a397-2b6ce7aa6b65'},
   'deployment': {'created_at': '2019-03-19T14:00:27.898Z',
    'deployment_id': '4d125552-83ee-456a-b1d4-3fd518743082',
    'deployment_rn': '',
    'deployment_type': 'online',
    'name': 'Spark German Risk Deployment - Final',
    'scoring_endpoint': {'request_headers': {'Content-Type': 'application/json'},
     'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/9e7215a4-16bb-464f-9e31-ff9270e66abb/deployments/4d125552-83ee-456a-b1d4-3fd518743082/online'},
    'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/9e7215a4-16bb-464f-9e31-ff9270e66abb/deployments/4d125552-83ee-456a-b1d4-3fd518743082'},
   'metrics

## Fairness monitoring

In [52]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.95),
                Feature("Age", majority=[[26,75]], minority=[[18,25]], threshold=0.95)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=1000,
            training_data=pd_data
        )

## Score the model again now that monitoring is configured

In [53]:
!rm german_credit_feed.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_feed.json

--2019-03-19 09:05:41--  https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3076547 (2.9M) [text/plain]
Saving to: ‘german_credit_feed.json’

100%[======================================>] 3,076,547   --.-K/s   in 0.08s   

2019-03-19 09:05:41 (35.4 MB/s) - ‘german_credit_feed.json’ saved [3076547/3076547]



Score 1000 randomly chosen records

In [54]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(1000):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)
print(scoring_response)

{'fields': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['less_0', 22, 'prior_payments_delayed', 'radio_tv', 4182, '100_to_500', '1_to_4', 3, 'male', 'none', 2, 'car_other', 29, 'none', 'own', 1, 'skilled', 1, 'none', 'yes', 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 1.0, 0.0, 0.0, [21, [0, 2, 3, 7, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 3.0, 1.0, 1.0, 22.0, 4182.0, 3.0, 2.0, 22.0, 29.0

In [55]:
subscription.get_details()

{'entity': {'asset': {'asset_id': 'b9473da0-9ae7-474a-a397-2b6ce7aa6b65',
   'asset_type': 'model',
   'created_at': '2019-03-19T14:00:27.876Z',
   'name': 'Spark German Risk Model - Final',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/9e7215a4-16bb-464f-9e31-ff9270e66abb/published_models/b9473da0-9ae7-474a-a397-2b6ce7aa6b65'},
  'asset_properties': {'categorical_fields': ['CheckingStatus',
    'CreditHistory',
    'LoanPurpose',
    'ExistingSavings',
    'EmploymentDuration',
    'Sex',
    'OthersOnLoan',
    'OwnsProperty',
    'InstallmentPlans',
    'Housing',
    'Job',
    'Telephone',
    'ForeignWorker'],
   'feature_fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'J

# Insert historical payloads

In [56]:
!rm payload_history*.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_1.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_2.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_3.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_4.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_5.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_6.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_7.json

--2019-03-19 09:06:09--  https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3817949 (3.6M) [text/plain]
Saving to: ‘payload_history_1.json’

100%[======================================>] 3,817,949   --.-K/s   in 0.1s    

2019-03-19 09:06:09 (34.6 MB/s) - ‘payload_history_1.json’ saved [3817949/3817949]

--2019-03-19 09:06:09--  https://raw.githubusercontent.com/emartensibm/german-credit/master/payload_history_2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3818645 (3.6M) [text/plain]
Saving to: ‘payload_history_2.json

In [57]:
historyDays = 7

In [58]:
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
import datetime
import time

for day in range(historyDays):
    print('Loading day {}'.format(day + 1))
    history_file = 'payload_history_' + str(day + 1) + '.json'
    with open(history_file) as f:
        payloads = json.load(f)
        hourly_records = int(len(payloads) / 24)
        index = 0
        for hour in range(24):
            recordsList = []
            for i in range(hourly_records):
                score_time = str(datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1))))
                recordsList.append(PayloadRecord(request=payloads[index]['request'], response=payloads[index]['response'], scoring_timestamp=score_time))
                index += 1
            subscription.payload_logging.store(records=recordsList)
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


In [59]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

f94e20cb-aa92-4636-97ce-08de6b714d02


In [60]:
performance_metrics_url = 'https://api.aiopenscale.cloud.ibm.com' + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

https://api.aiopenscale.cloud.ibm.com/v1/data_marts/f94e20cb-aa92-4636-97ce-08de6b714d02/metrics


## Insert historical fairness metrics

In [61]:
!rm fairness_history.json
!wget https://raw.githubusercontent.com/emartensibm/german-credit/master/fairness_history.json

--2019-03-19 09:06:59--  https://raw.githubusercontent.com/emartensibm/german-credit/master/fairness_history.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 877247 (857K) [text/plain]
Saving to: ‘fairness_history.json’

100%[======================================>] 877,247     --.-K/s   in 0.04s   

2019-03-19 09:07:00 (18.9 MB/s) - ‘fairness_history.json’ saved [877247/877247]



In [62]:
import random
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('fairness_history.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': random.choice(payloads)
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Insert historical quality metrics

In [63]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.7,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.7
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Insert historical performance metrics

In [64]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(60, 600)
        score_resp = random.uniform(60, 300)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Configure Explainability

In [65]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(training_data=pd_data)

In [66]:
subscription.explainability.get_details()

{'enabled': True,
 'parameters': {'training_statistics': {'base_values': {'0': 'no_checking',
    '1': 21.0,
    '10': 3.0,
    '11': 'savings_insurance',
    '12': 36.0,
    '13': 'none',
    '14': 'own',
    '15': 1.0,
    '16': 'skilled',
    '17': 1.0,
    '18': 'none',
    '19': 'yes',
    '2': 'prior_payments_delayed',
    '3': 'car_new',
    '4': 3238.5,
    '5': 'less_100',
    '6': '1_to_4',
    '7': 3.0,
    '8': 'male',
    '9': 'none'},
   'categorical_columns': ['CheckingStatus',
    'CreditHistory',
    'LoanPurpose',
    'ExistingSavings',
    'EmploymentDuration',
    'Sex',
    'OthersOnLoan',
    'OwnsProperty',
    'InstallmentPlans',
    'Housing',
    'Job',
    'Telephone',
    'ForeignWorker'],
   'categorical_columns_encoding_mapping': {'0': ['0_to_200',
     'greater_200',
     'less_0',
     'no_checking'],
    '11': ['car_other', 'real_estate', 'savings_insurance', 'unknown'],
    '13': ['bank', 'none', 'stores'],
    '14': ['free', 'own', 'rent'],
    '16': 

## Run fairness monitor

Kick off a fairness monitor run on current data. Depending on how fast the monitor runs, the table may not contain the most recent results.

In [67]:
run_details = subscription.fairness_monitoring.run()

In [68]:
subscription.fairness_monitoring.show_table()

2019-03-19 13:07:07+00:00,Sex,female,True,0.918,72.5,9e7215a4-16bb-464f-9e31-ff9270e66abb,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,4d125552-83ee-456a-b1d4-3fd518743082,
2019-03-19 13:07:07+00:00,Age,"[18, 25]",False,1.025,83.65650969529086,9e7215a4-16bb-464f-9e31-ff9270e66abb,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,4d125552-83ee-456a-b1d4-3fd518743082,
2019-03-19 14:08:43.777913+00:00,Sex,female,True,0.947,72.7,9e7215a4-16bb-464f-9e31-ff9270e66abb,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,4d125552-83ee-456a-b1d4-3fd518743082,
2019-03-19 14:08:43.777913+00:00,Age,"[18, 25]",False,0.989,79.24528301886792,9e7215a4-16bb-464f-9e31-ff9270e66abb,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,4d125552-83ee-456a-b1d4-3fd518743082,
2019-03-19 14:11:48.569594+00:00,Sex,female,True,0.947,72.7,9e7215a4-16bb-464f-9e31-ff9270e66abb,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,4d125552-83ee-456a-b1d4-3fd518743082,
2019-03-19 14:11:48.569594+00:00,Age,"[18, 25]",False,0.989,79.24528301886792,9e7215a4-16bb-464f-9e31-ff9270e66abb,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,b9473da0-9ae7-474a-a397-2b6ce7aa6b65,4d125552-83ee-456a-b1d4-3fd518743082,


## Additional data to help debugging

In [69]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)
print('Scoring URL:', credit_risk_scoring_endpoint)

Datamart: f94e20cb-aa92-4636-97ce-08de6b714d02
Model: b9473da0-9ae7-474a-a397-2b6ce7aa6b65
Deployment: 4d125552-83ee-456a-b1d4-3fd518743082
Binding: 9e7215a4-16bb-464f-9e31-ff9270e66abb
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/9e7215a4-16bb-464f-9e31-ff9270e66abb/deployments/4d125552-83ee-456a-b1d4-3fd518743082/online


## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [70]:
payload_data = subscription.payload_logging.get_table_content(limit=60)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability'])

,scoring_id,predictedLabel,probability
0,dee896e754a91d3b0ef6428a3d190bc1-147,Risk,"[0.39529250215656087, 0.6047074978434391]"
1,dee896e754a91d3b0ef6428a3d190bc1-150,Risk,"[0.16901307178954633, 0.8309869282104536]"
2,dee896e754a91d3b0ef6428a3d190bc1-142,No Risk,"[0.7983854192170071, 0.20161458078299285]"
3,dee896e754a91d3b0ef6428a3d190bc1-146,No Risk,"[0.6076710233097418, 0.39232897669025835]"
4,dee896e754a91d3b0ef6428a3d190bc1-149,No Risk,"[0.6761969235818277, 0.32380307641817224]"
5,dee896e754a91d3b0ef6428a3d190bc1-15,No Risk,"[0.6756922948373565, 0.3243077051626435]"
6,dee896e754a91d3b0ef6428a3d190bc1-141,No Risk,"[0.845919183590898, 0.15408081640910204]"
7,dee896e754a91d3b0ef6428a3d190bc1-133,No Risk,"[0.5266038757997153, 0.47339612420028454]"
8,dee896e754a91d3b0ef6428a3d190bc1-145,No Risk,"[0.920162863435178, 0.07983713656482216]"
9,dee896e754a91d3b0ef6428a3d190bc1-121,No Risk,"[0.7520103580613784, 0.2479896419386216]"


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

## Next steps

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and cal be deleted if necessary.